# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
#грузим dataset и смотрим на общие вид данных
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
#смотрим где и какое колличество пропусков
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [2]:
#доля пропущенных значений
print(f"Доля пропущенных значений = {df['total_income'].isnull().sum()/len(df):.2%}")

Доля пропущенных значений = 10.10%


In [3]:
#посмотрим что за пустые значения
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


В двух столбцах 'days_employed' и 'total_income' 10.1% данных пустые значения - NaN, скорее всего у этих людей нет официально подтвержденного дохода и соответсвенно стажа. Заполним пропуски медианным значением - экстремальные выбросы в выборке не искажают медиану в отличие от среднего, поэтому медиана является лучшим показателем тенденции, чем среднее значение.

### Шаг 2.1 Заполнение пропусков

#### Столбец 'total_income'

Найдем медианное значение столбца 'total_income' и заполним им пропуски:

In [4]:
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)
#проверка
df['total_income'].isna().sum()

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

#### Столбец 'days_employed', проверим его на аномалии и обработаем их при необходимости

In [5]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Стаж отрицательным быть не может, заменим отрицательные значения на положительные:

In [6]:
#функция для замены отрицательных значений на положительные
def plus_minus(number):
    if number < 0:
        number *= -1
        return number
    else:
        return number
#применим функцию к столбцу days_employed
df['days_employed'] = df['days_employed'].apply(plus_minus)
#найдем медиану столбца days_employed и заполним ей пропуски
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['days_employed'].describe()

count     21525.000000
mean      60378.032733
std      133257.558514
min          24.141633
25%        1025.608174
50%        2194.220567
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Скорее всего в минус превратилось использованное при вводе вручную "тире". В данных также есть очень большие значения не соответсвующие реально возможному стажу, но поскольку стаж, как критерий, не фигурирует в вопросах поставленных в задаче, то на данном этапе целесообразно оставить данные как есть

#### Столбец 'children'

In [7]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Некорректные значения "20" и "-1"; их колличество незначительно, сделаем визуальный анализ:

In [8]:
df[df['children'] == 20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,2194.220567,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,145017.937533,профильное образование
3396,20,2194.220567,56,высшее,0,женат / замужем,0,F,компаньон,0,145017.937533,высшее образование
3671,20,913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


In [9]:
df[df['children'] == -1].tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17095,-1,2809.693200,34,среднее,1,женат / замужем,0,F,сотрудник,0,182543.890127,на покупку подержанного автомобиля
17429,-1,895.379738,37,среднее,1,женат / замужем,0,F,компаньон,0,214814.018780,жилье
17657,-1,4571.957475,41,среднее,1,женат / замужем,0,F,сотрудник,0,122105.415823,операции со своей недвижимостью
18219,-1,3575.215641,33,среднее,1,женат / замужем,0,F,сотрудник,0,128362.023879,высшее образование
19100,-1,617.246968,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,122205.497527,строительство жилой недвижимости
19366,-1,372.034749,43,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,155588.766795,сделка с подержанным автомобилем
19417,-1,350340.760224,28,среднее,1,в разводе,3,F,пенсионер,0,52872.993654,автомобили
20393,-1,355157.107212,69,среднее,1,Не женат / не замужем,4,F,пенсионер,0,116521.045858,операции с недвижимостью
20717,-1,661.822321,32,высшее,0,женат / замужем,0,F,сотрудник,0,137405.384917,покупка жилой недвижимости
21140,-1,1422.668059,44,среднее,1,женат / замужем,0,F,компаньон,0,169562.091999,операции со своей недвижимостью


Очевидных закономерностей нет. Вполне вероятно, что "-1" - это "тире" и "1", а "20" - двойка и "нечаянный" ноль на NUMPAD-е, но поскольку суммарно строки со значениями "20" и "-1" занимают меньше 0,5% от общих данных, то гадать не будем и просто их отфильтруем:

In [10]:
df = df[(df['children'] != 20) & (df['children'] != -1)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


#### Столбец 'dob_years'

In [11]:
df['dob_years'].value_counts().sort_index()

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    461
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

100 значений с возрастом "0", очевидная ошибка данных, сделаем визуальный анализ:

In [12]:
df[df['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,2194.220567,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,145017.937533,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Очевидных закономерностей нет; поскольку возраст, как критерий, не фигурирует в вопросах поставленных в задаче, то на данном этапе целесообразно оставить данные как есть

#### Столбец 'education'

In [13]:
df['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Если привести данные к одному регистру - будет сделано в пункте 1.0.5, то получится 5 категорий

#### Столбец 'education_id'

In [14]:
df['education_id'].value_counts()

1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

5 категорий, совпадает со столбцом "education", данные корректны

#### Столбцы 'family_status' и 'family_id'

In [15]:
df['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [16]:
df['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

Колличество в категориях совпадает, данные корректны

#### Столбец 'gender'

In [17]:
df['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Одна строка с некорректным значением, посмотрим на нее:

In [18]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Одна строка на 21k+ данных скорее всего ни на что не повлияет, поэтому "ткнем пальцем в небо" - 24 года, без детей, высокий доход, покупка недвижимости - вероятно это мужчина, исправим строку:

In [19]:
df.loc[df['gender']=='XNA','gender'] = 'M'
df['gender'].value_counts()

F    14154
M     7248
Name: gender, dtype: int64

#### Столбец 'income_type'

In [20]:
df['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Данные корректны

#### Столбец 'debt'

In [21]:
df['debt'].value_counts()

0    19670
1     1732
Name: debt, dtype: int64

Данные корректны

#### Столбец 'purpose'

In [22]:
df['purpose'].value_counts()
#len(set(df['purpose']))

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

38 по разному записанных целей кредита, которые можно собрать в 4 категории - "недвижимость", "авто", "образование", "свадьба" - будет сделано в пункте 1.0.8

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце 'total_income' на целочисленный:

In [23]:
df['total_income'] = df['total_income'].astype('int')
#проверка
df['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

### Шаг 2.4. Удаление дубликатов.

Обработаем неявные дубликаты в столбце 'education', приведем данные к одному регистру:

In [24]:
df['education'] = df['education'].str.lower()
#проверка
df['education'].head(10)

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
5     высшее
6     высшее
7    среднее
8     высшее
9    среднее
Name: education, dtype: object

Проверим, совпадают ли теперь данные в столбцах 'education' и 'education_id':

In [25]:
display(df['education'].value_counts())
display(df['education_id'].value_counts())

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

Данные совпадают, неявные дубликаты убраны; найдем явные дубликаты и удалим их: 

In [26]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Дубликаты появляются из-за человеческих ошибок при вводе, например при использовании разных регистров или из-за повторного обращении одного и того же заемщика к разным сотрудникам/офисам/отделениям. Неявные дубликаты трудно обнаружить - визуально и с использовнием "set" или "unique"; для явных дубликатов использовался метод "duplicated"

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых - каждому уникальному значению из 'education' соответствует уникальное значение 'education_id' и каждому уникальному значению из 'family_status' соответствует уникальное значение 'family_status_id':

In [27]:
education_new = df[['education_id', 'education']]
family_status_new = df[['family_status_id', 'family_status']]
#удалим дубликаты из словарей
education_new = education_new.drop_duplicates().reset_index(drop=True)
family_status_new = family_status_new.drop_duplicates().reset_index(drop=True)
#проверим что получилось
display(education_new.head(10))
display(family_status_new.head(10))

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удалим из исходного датафрейма столбцы 'education' и 'family_status':

In [28]:
df = df.drop(columns=['education', 'family_status'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21331 non-null  int64  
 1   days_employed     21331 non-null  float64
 2   dob_years         21331 non-null  int64  
 3   education_id      21331 non-null  int64  
 4   family_status_id  21331 non-null  int64  
 5   gender            21331 non-null  object 
 6   income_type       21331 non-null  object 
 7   debt              21331 non-null  int64  
 8   total_income      21331 non-null  int64  
 9   purpose           21331 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

Создадим столбец 'total_income_category', на основании диапазонов: "0-30000" — 'E'; "30001–50000" — 'D'; "50001–200000" — 'C'; "200001–1000000" — 'B'; "1000001 и выше" — 'A' в столбце 'total_income'

In [29]:
#функция для категорий дохода
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if 30000 < total_income <= 50000:
        return 'D'
    if 50000 < total_income <= 200000:
        return 'C'
    if 200000 < total_income <= 1000000:
        return 'B'
    return 'A'
#создадим и заполним столбец 'total_income_category'
df['total_income_category'] = df['total_income'].apply(total_income_category)
#проверка
df['total_income_category'].value_counts()

C    15922
B     5013
D      349
A       25
E       22
Name: total_income_category, dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education_id           21331 non-null  int64  
 4   family_status_id       21331 non-null  int64  
 5   gender                 21331 non-null  object 
 6   income_type            21331 non-null  object 
 7   debt                   21331 non-null  int64  
 8   total_income           21331 non-null  int64  
 9   purpose                21331 non-null  object 
 10  total_income_category  21331 non-null  object 
dtypes: float64(1), int64(6), object(4)
memory usage: 1.8+ MB


### Шаг 2.7. Категоризация целей кредита.

Из данных столбца 'purpose' сформируем новый столбец 'purpose_category' в который войдут следующие категории - 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования':

In [31]:
#вызываем коллекцию Counter и лемматизатор для русских слов
#from collections import Counter
from pymystem3 import Mystem
m = Mystem()

#функция для распределения целей кредита по новым категориям
def new_purpose(purpose):
    lemmas_purpose = m.lemmatize(purpose)
    if ('недвижимость' in lemmas_purpose) | ('жилье' in lemmas_purpose):
        return 'операции с недвижимостью'
    if 'автомобиль' in lemmas_purpose:
        return 'операции с автомобилем'
    if 'образование' in lemmas_purpose:
        return 'получение образования'
    if 'свадьба' in lemmas_purpose:
        return 'проведение свадьбы'
    return''
#применим функцию new_purpose к столбцу 'purpose'
df['purpose_category'] = df['purpose'].apply(new_purpose)
#проверка новового столбца 'purpose_category', есть ли значения без категории, есть ли дубликаты
display(df['purpose_category'].value_counts())
display(df[df['purpose_category'] == ""])
df.info()
df.duplicated().sum()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education_id           21331 non-null  int64  
 4   family_status_id       21331 non-null  int64  
 5   gender                 21331 non-null  object 
 6   income_type            21331 non-null  object 
 7   debt                   21331 non-null  int64  
 8   total_income           21331 non-null  int64  
 9   purpose                21331 non-null  object 
 10  total_income_category  21331 non-null  object 
 11  purpose_category       21331 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


0

### Ответы на вопросы.

##### Вопрос 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [32]:
#функция для подсчета результата в процентах
def percentage(values):
    return f'{(values.sum() / values.count()):.2%}'
#строим сводную таблицу для ответа на вопрос
child_debt = df.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', percentage])
child_debt

,sum,count,percentage
,debt,debt,debt
children,,,
0,1063,14091,7.54%
1,444,4808,9.23%
2,194,2052,9.45%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


##### Вывод 1:

С увеличением количества детей растет просрочка, и  хотя в нашем исследовании люди с 3-мя детьми чаще платят в срок чем люди с 1-м ребенком это, вероятно, результат маленькой выборки. Бездетные платят по кредиту заметно аккуратнее, чем люди с детьми.

##### Вопрос 2:  Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
#для удобства вернем 'family_status' обратно в таблицу метод merge()
df_merge = df.merge(family_status_new, on='family_status_id', how='left')
fam_status_debt = df_merge.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', percentage])
fam_status_debt

,sum,count,percentage
,debt,debt,debt
family_status,,,
Не женат / не замужем,273,2796,9.76%
в разводе,84,1189,7.06%
вдовец / вдова,63,951,6.62%
гражданский брак,385,4134,9.31%
женат / замужем,927,12261,7.56%


##### Вывод 2:

Если семейные отношения оформлены (или были офрмлены) официально, то просрочка заметно ниже, чем у людей без отношений или просто живущих вместе.

##### Вопрос 3:  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
income_debt = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', percentage])
income_debt

,sum,count,percentage
,debt,debt,debt
total_income_category,,,
A,2,25,8.00%
B,354,5013,7.06%
C,1353,15922,8.50%
D,21,349,6.02%
E,2,22,9.09%


##### Вывод 3:

По категриям "B" и "C" - чем выше доходы, тем меньше просрочка. По категориям "A", "D", "E" данных мало, поэтому скорее всего вывод по ним будет некорректен. 

##### Вопрос 4:  Как разные цели кредита влияют на его возврат в срок?

In [35]:
purpose_debt = df.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', percentage])
purpose_debt

,sum,count,percentage
,debt,debt,debt
purpose_category,,,
операции с автомобилем,400,4279,9.35%
операции с недвижимостью,780,10751,7.26%
получение образования,369,3988,9.25%
проведение свадьбы,183,2313,7.91%


##### Вывод 4:

Покупка недвижимости в кредит для большинства людей - серъезное, взвешенное решение, как результат - самый низкий процент просрочки. Свадьба - начало семейной жизни, осмысленное решение для обоих, что подразумевает дисциплину, итог - второе место по качеству кредитного портфеля. Образование и автомобиль в кредит - это, как правило, признак невысоких доходов, логично, что просрочка самая высокая.

## Общий вывод:

Низкие доходы и высокие расходы (колличество детей) приводят к более высокой просрочке. Семейный статус влияет на дисциплину оплаты кредита. Недвижимость - самая "безопасная" цель для банков при выдаче кредита. Идеальный заемщик - человек без детей, состоящий/состоявший в официальных отношениях с доходами выше среднего и берущий кредит на недвижимость.